### 1) Setup & imports

In [ ]:
%pip install -r requirements.txt

In [ ]:
# from pathlib import Path
# from importlib import reload

# # If running in notebook, make project path importable
# import sys
# proj = Path.cwd().parent  # adjust if needed
# if str(proj) not in sys.path:
#     sys.path.append(str(proj))

# from src import constants as C
# from src import train as T
# from src import infer as I
# from src import model as M
# reload(C); reload(T); reload(I); reload(M);

# Download a set of LR-HR images for training
%time python -m src.pipelines.get_image_pairs -n 500

### 2) (Optional) Download a generic ESRGAN x4 pretrained weight (RRDBNet)

In [ ]:
# # Many public weights exist. Example: you can add your own path to a .pth here for finetuning/inference.
# WEIGHT_PATH = None  # '/content/drive/MyDrive/weights/ESRGAN_x4.pth'

### 3) Train / Finetune (EDSR x4)

In [ ]:
# Ensure you have paired LR/HR images (same filenames) under data/LR and data/HR.
# Training will crop random aligned patches from HR and downsample to create LR (robust even if LR not perfectly registered).
# You can also use your own LR images for training by changing it on the `dataset.py` file
%time python -m src.train

### 4) Inference on a dir of LR images

In [ ]:
# from pathlib import Path
# sample_imgs = list((C.LR_DIR).glob('*.png'))[:3]
# I.run_inference(sample_imgs, model_name=C.MODEL_NAME, scale=C.SCALE, ckpt_path=str((C.CKPT_DIR / f'best_{C.MODEL_NAME}_x{C.SCALE}.pth')) if (C.CKPT_DIR / f'best_{C.MODEL_NAME}_x{C.SCALE}.pth').exists() else None, progressive_to=8)

#   python -m src.infer data/LR/36.22243_-5.84406_test.png data/LR/36.31215_-5.93888_test.png --model edsr --scale 4 --weigths best_edsr_x4.pth --progressive-to 8
#   python -m src.infer data/LR/*.png --model edsr --scale 4 --weigths best_edsr_x4.pth
%time python -m src.infer data/LR --model edsr --scale 4 --weigths best_edsr_x4.pth --progressive-to 8


### 5) Metrics on held-out pairs (quick)

In [ ]:
# import numpy as np
# from src.dataset import PairedImageDataset
# from src.utils import psnr, ssim
# import torch

# val_ds = PairedImageDataset(C.LR_DIR, C.HR_DIR, scale=C.SCALE, augment=False)

# import random
# idxs = random.sample(range(len(val_ds)), min(10, len(val_ds)))

# import torchvision.transforms.functional as TF
# from src.model import build_model, ModelConfig
# model = build_model(ModelConfig(name=C.MODEL_NAME, scale=C.SCALE, pretrained=str(C.CKPT_DIR / f'best_{C.MODEL_NAME}_x{C.SCALE}.pth') if (C.CKPT_DIR / f'best_{C.MODEL_NAME}_x{C.SCALE}.pth').exists() else None)).eval()
# model = model.cuda() if torch.cuda.is_available() else model

# scores = []
# for i in idxs:
#     batch = val_ds[i]
#     lr = batch['lr'].unsqueeze(0)
#     hr = batch['hr'].unsqueeze(0)
#     if torch.cuda.is_available():
#         lr, hr = lr.cuda(), hr.cuda()
#     with torch.no_grad():
#         sr = model(lr).clamp(0,1)
#     sr_np = (sr[0].permute(1,2,0).cpu().numpy()*255).astype('uint8')